In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import cv2
import csv
import numpy as np
from time import time
import keras
from tensorflow.keras import utils
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras.applications import VGG19
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow # for image display
#%matplotlib inline

In [ ]:
input_size = (32, 32)
dir = '/content/drive/MyDrive/CS231_Hoang_Nam_Vy'

In [ ]:
def load_data(path, input_size):
    images = []
    labels = []
    cnt = 0
    for folder in os.listdir(os.path.join(path, 'DataReg')):
        cur_path = os.path.join(path, 'DataReg', folder)
        for file_name in os.listdir(cur_path):
            image = cv2.imread(os.path.join(cur_path, file_name))
            image = cv2.resize(image, input_size)
            images.append(image)
            labels.append(int(folder))
    return images, labels

In [ ]:
images, labels = load_data(dir + "/", input_size=input_size)

In [ ]:
def split_train_val_test_data(images, labels):

    # Chuẩn hoá dữ liệu images và labels
    images = np.array(images)
    labels = keras.utils.np_utils.to_categorical(labels)

    # Nhào trộn dữ liệu ngẫu nhiên
    randomize = np.arange(len(images))
    np.random.shuffle(randomize)
    X = images[randomize]
    print("X=", X.shape)
    y = labels[randomize]

    # Chia dữ liệu theo tỷ lệ 60% train và 40% còn lại cho val và test
    train_size = int(X.shape[0] * 0.6)
    X_train, X_val = X[:train_size], X[train_size:]
    y_train, y_val = y[:train_size], y[train_size:]

    val_size = int(X_val.shape[0] * 0.5) # 50% của phần 40% bên trên
    X_val, X_test = X_val[:val_size], X_val[val_size:]
    y_val, y_test = y_val[:val_size], y_val[val_size:]

    return X_train, y_train, X_val, y_val, X_test, y_test

In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test = split_train_val_test_data(images, labels)

X= (11000, 32, 32, 3)


In [ ]:
classes = 7
batch = 256
epochs = 3
learning_rate = 0.0001

def results(model):
  adam = Adam(lr=learning_rate)

  model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

  start = time()
  history = model.fit(X_train, y_train, batch_size=batch, epochs=epochs, validation_data=(X_val, y_val), shuffle = True, verbose=1)
  train_time = time() - start

  model.summary()

  plt.figure(figsize=(12, 12))
  plt.subplot(3, 2, 1)
  plt.plot(history.history['accuracy'], label = 'train_accuracy')
  plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
  plt.xlabel('epoch')
  plt.ylabel('accuracy')
  plt.legend()
  plt.subplot(3, 2, 2)
  plt.plot(history.history['loss'], label = 'train_loss')
  plt.plot(history.history['val_loss'], label = 'val_loss')
  plt.xlabel('epoch')
  plt.ylabel('accuracy')
  plt.legend()
  plt.show()

  start = time()
  test_loss, test_acc = model.evaluate(X_val, y_val)
  test_time = time() - start
  print('\nTrain time: ', train_time)
  print('Test accuracy:', test_acc)
  print('Test loss:', test_loss)
  print('Test time: ', test_time)

In [ ]:
model = Sequential()
model.add(VGG19(weights='imagenet', include_top=False, input_shape=(32,32,3)))
model.add(BatchNormalization())
model.add(ReLU())
model.add(Flatten())
model.add(Dense(7))
model.add(Softmax())

results(model)

model.save('/content/drive/MyDrive/CS231_Hoang_Nam_Vy/vgg19.h5')

In [ ]:
# Kiểm tra model với dữ liệu mới
model = keras.models.load_model("C:/Users/ADMIN/Desktop/VGG19/vgg19.h5")

label_name = [
'Cấm đi ngược chiều', 
'Cấm đỗ hoặc cấm dừng', 
'Cấm rẽ', 
'Giới hạn tốc độ tối đa', 
'Biển cấm', 
'Biển báo nguy hiểm',
'Biển hiệu lệnh và chỉ dẫn'
]

In [ ]:
def printLabel(img):
    img = cv2.resize(img, input_size)
    img = np.array(img)
    img = np.expand_dims(img, axis = 0)

    #Predict the inputs on the model
    predict_img = model.predict(img)
    predict_img = predict_img[0]
    print(label_name[np.argmax(predict_img)])
start = time()
printLabel(cv2.imread('C:/Users/ADMIN/Desktop/VGG19/Test/x.png'))
test_time = time() - start
print('Test time: ', test_time)